In [1]:
%matplotlib inline
import scipy
import numpy as np
import pandas as pd
import pandas.io.formats.style
import seaborn as sns
import statsmodels as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
from IPython.display import HTML
import linearmodels as lm
from linearmodels import PanelOLS
import warnings
warnings.filterwarnings("ignore")
import math
from linearmodels.panel import compare
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.api as sm_api


In [2]:
# path="/Users/mengxi/Desktop/financial research/github final/"
# data=pd.read_stata(path+"211231_total_and_by_industry_data_county.dta")


path="/Users/effie/Desktop/term3/research_module/data/"
data=pd.read_stata(path+"county_clean/total_and_by_industry_data_county.dta")
data=data.drop(columns=['index'])


data


,year,Gebietseinheit,EU_Code,Regional_key,Land,resident,ln_gdp,gdp_level,gdp_per_inhabitant,gdp_per_employee,...,toas_ratio,emp_agr_weight,emp_man_weight,emp_ser_weight,gva_agr_weight,gva_man_weight,gva_ser_weight,vow_agr_weight,vow_man_weight,vow_ser_weight
0,2004,"Stuttgart, Landeshauptstadt, Stadtkreis",DE111,08111,BW,573.955017,-1.059015,40680.228,70877.0,85811.0,...,1.0,0.002413,0.213166,0.784421,0.000634,0.324826,0.674540,0.002775,0.223713,0.773512
1,2004,"Böblingen, Landkreis",DE112,08115,BW,366.351990,-3.269339,14478.332,39520.0,67825.0,...,1.0,0.004872,0.422414,0.572714,0.001776,0.505710,0.492514,0.006840,0.446232,0.546928
2,2004,"Esslingen, Landkreis",DE113,08116,BW,503.649994,-0.074150,15205.272,30190.0,61149.0,...,1.0,0.006507,0.376005,0.617488,0.002375,0.408458,0.589167,0.008329,0.397011,0.594661
3,2004,"Göppingen, Landkreis",DE114,08117,BW,255.177994,-0.930964,6158.588,24134.0,54533.0,...,1.0,0.013096,0.400742,0.586162,0.006225,0.432973,0.560802,0.019670,0.424832,0.555498
4,2004,"Ludwigsburg, Landkreis",DE115,08118,BW,504.459991,0.341932,15851.325,31422.0,67176.0,...,1.0,0.014722,0.341771,0.643507,0.005007,0.376126,0.618868,0.018984,0.361308,0.619708
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3586,2012,Saale-Orla-Kreis,DEG0K,16075,TH,84.847000,5.540099,1955.848,23051.0,48013.0,...,NaN,0.044653,0.428515,0.526831,0.031515,0.474068,0.494417,NaN,NaN,NaN
3587,2012,"Greiz, Landkreis",DEG0L,16076,TH,103.880997,2.040082,1854.354,17851.0,46776.0,...,0.0,0.037358,0.373862,0.588780,0.031252,0.365997,0.602751,NaN,NaN,NaN
3588,2012,"Altenburger Land, Landkreis",DEG0M,16077,TH,95.289001,-1.616072,1689.681,17732.0,45168.0,...,1.0,0.028015,0.338902,0.633083,0.023566,0.340479,0.635954,NaN,NaN,NaN
3589,2012,"Eisenach, Kreisfreie Stadt",DEG0N,16056,TH,NaN,NaN,NaN,NaN,NaN,...,0.0,0.002846,0.296681,0.700473,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
data["No_ratio"].describe()

count    2691.000000
mean        0.286676
std         0.340037
min         0.000000
25%         0.000000
50%         0.166667
75%         0.500000
max         1.000000
Name: No_ratio, dtype: float64

In [4]:
data["toas_ratio"].describe()

count    2691.000000
mean        0.625744
std         0.476284
min         0.000000
25%         0.000000
50%         1.000000
75%         1.000000
max         1.146924
Name: toas_ratio, dtype: float64

In [5]:
query_data = data.query('No_ratio>0.286')
data.loc[:,'county_affected_no'] = data.index.map(lambda x : 1 if x in query_data.index else 0)

query_data = data.query('toas_ratio>0.627')
data.loc[:,'county_affected_toas'] = data.index.map(lambda x : 1 if x in query_data.index else 0)

query_data = data.query('year>2008')
data.loc[:,'year_post'] = data.index.map(lambda x : 1 if x in query_data.index else 0)

data["affected_after_toas"] = data.county_affected_toas*data.year_post
data["affected_ratio_after"]=data.toas_ratio*data.year_post


data

,year,Gebietseinheit,EU_Code,Regional_key,Land,resident,ln_gdp,gdp_level,gdp_per_inhabitant,gdp_per_employee,...,gva_man_weight,gva_ser_weight,vow_agr_weight,vow_man_weight,vow_ser_weight,county_affected_no,county_affected_toas,year_post,affected_after_toas,affected_ratio_after
0,2004,"Stuttgart, Landeshauptstadt, Stadtkreis",DE111,08111,BW,573.955017,-1.059015,40680.228,70877.0,85811.0,...,0.324826,0.674540,0.002775,0.223713,0.773512,1,1,0,0,0.0
1,2004,"Böblingen, Landkreis",DE112,08115,BW,366.351990,-3.269339,14478.332,39520.0,67825.0,...,0.505710,0.492514,0.006840,0.446232,0.546928,0,1,0,0,0.0
2,2004,"Esslingen, Landkreis",DE113,08116,BW,503.649994,-0.074150,15205.272,30190.0,61149.0,...,0.408458,0.589167,0.008329,0.397011,0.594661,0,1,0,0,0.0
3,2004,"Göppingen, Landkreis",DE114,08117,BW,255.177994,-0.930964,6158.588,24134.0,54533.0,...,0.432973,0.560802,0.019670,0.424832,0.555498,1,1,0,0,0.0
4,2004,"Ludwigsburg, Landkreis",DE115,08118,BW,504.459991,0.341932,15851.325,31422.0,67176.0,...,0.376126,0.618868,0.018984,0.361308,0.619708,0,1,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3586,2012,Saale-Orla-Kreis,DEG0K,16075,TH,84.847000,5.540099,1955.848,23051.0,48013.0,...,0.474068,0.494417,NaN,NaN,NaN,0,0,1,0,NaN
3587,2012,"Greiz, Landkreis",DEG0L,16076,TH,103.880997,2.040082,1854.354,17851.0,46776.0,...,0.365997,0.602751,NaN,NaN,NaN,0,0,1,0,0.0
3588,2012,"Altenburger Land, Landkreis",DEG0M,16077,TH,95.289001,-1.616072,1689.681,17732.0,45168.0,...,0.340479,0.635954,NaN,NaN,NaN,1,1,1,1,1.0
3589,2012,"Eisenach, Kreisfreie Stadt",DEG0N,16056,TH,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0,0,1,0,0.0


In [6]:
data['county_affected_toas'].value_counts()

0    1908
1    1683
Name: county_affected_toas, dtype: int64

In [7]:
data["affected_ratio_after"].value_counts()

0.000000    1903
1.000000     704
0.151608       4
0.991118       4
0.014877       4
0.084142       4
0.705943       4
0.194287       4
0.167133       4
0.996335       4
0.873121       4
0.102946       4
0.632758       4
0.634103       4
0.240669       4
0.999252       4
0.836010       4
1.146924       4
0.427839       4
0.167701       4
0.840457       4
0.028159       4
0.862076       4
Name: affected_ratio_after, dtype: int64

In [8]:


data["ln_res"]=np.log(data["resident"]*1000)

data["ln_gdp_level"] = np.log(data.gdp_level*1000000)
data["ln_gdp_per_inhabitant"] = np.log(data.gdp_per_inhabitant)
data["ln_gdp_per_employee"] = np.log(data.gdp_per_employee)
data["ln_gdp_per_hour_per_employed"] = np.log(data.gdp_per_hour_per_employed)



data["ln_gva_level"] = np.log(data.gva_level*1000000)
data["ln_gva_agr"] = np.log(data.gva_agr*1000000)
data["ln_gva_man"] = np.log(data.gva_man*1000000)
data["ln_gva_ser"] = np.log(data.gva_ser*1000000)
data["ln_gva_ser_fin"] = np.log(data.gva_ser_fin*1000000)
data["ln_gva_ser_trade"] = np.log(data.gva_ser_trade*1000000)
data["ln_gva_ser_public"] = np.log(data.gva_ser_public*1000000)

data["ln_emp"]=np.log(data["employee"]*1000)

data["ln_emp_ser_fin"] = np.log(data.emp_ser_fin*1000)
data["ln_emp_ser_trade"] = np.log(data.emp_ser_trade*1000)
data["ln_emp_ser_public"] = np.log(data.emp_ser_public*1000)
data["ln_emp_agr"]=np.log(data["emp_agr"]*1000)
data["ln_emp_man"]=np.log(data["emp_man"]*1000)
data["ln_emp_ser"]=np.log(data["emp_ser"]*1000)

data["ln_vow"]=np.log(data["volume_of_work"]*1000000)

data["ln_vow_ser_fin"] = np.log(data.vow_ser_fin*1000000)
data["ln_vow_ser_trade"] = np.log(data.vow_ser_trade*1000000)
data["ln_vow_ser_public"] = np.log(data.vow_ser_public*1000000)
data["ln_vow_agr"]=np.log(data["vow_agr"]*1000000)
data["ln_vow_man"]=np.log(data["vow_man"]*1000000)
data["ln_vow_ser"]=np.log(data["vow_ser"]*1000000)

data["ln_fixed_total"] = np.log(data.fixed_total*1000000)
data["ln_fixed_ser_fin"] = np.log(data.fixed_ser_fin*1000000)
data["ln_fixed_ser_fin"] = np.log(data.fixed_ser_fin*1000000)
data["ln_fixed_ser_trade"] = np.log(data.fixed_ser_trade*1000000)
data["ln_fixed_ser_public"] = np.log(data.fixed_ser_public*1000000)
data["ln_fixed_agr"]=np.log(data["fixed_agr"]*1000000)
data["ln_fixed_man"]=np.log(data["fixed_man"]*1000000)
data["ln_fixed_ser"]=np.log(data["fixed_ser"]*1000000)

df_fe = data.set_index(['Regional_key','year'])
df_fe

,,Gebietseinheit,EU_Code,Land,resident,ln_gdp,gdp_level,gdp_per_inhabitant,gdp_per_employee,gdp_per_hour_per_employed,ln_gva,...,ln_vow_agr,ln_vow_man,ln_vow_ser,ln_fixed_total,ln_fixed_ser_fin,ln_fixed_ser_trade,ln_fixed_ser_public,ln_fixed_agr,ln_fixed_man,ln_fixed_ser
Regional_key,year,,,,,,,,,,,,,,,,,,,,,
08111,2004,"Stuttgart, Landeshauptstadt, Stadtkreis",DE111,BW,573.955017,-1.059015,40680.228,70877.0,85811.0,58.340000,-0.712461,...,14.475618,18.865405,20.105983,24.950936,24.075167,22.963721,22.949218,20.168262,23.756601,24.578018
08115,2004,"Böblingen, Landkreis",DE112,BW,366.351990,-3.269339,14478.332,39520.0,67825.0,47.110001,-2.930522,...,14.558400,18.736418,18.939897,24.950936,24.075167,22.963721,22.949218,20.168262,23.756601,24.578018
08116,2004,"Esslingen, Landkreis",DE113,BW,503.649994,-0.074150,15205.272,30190.0,61149.0,42.700001,0.275855,...,14.902725,18.766969,19.170997,24.950936,24.075167,22.963721,22.949218,20.168262,23.756601,24.578018
08117,2004,"Göppingen, Landkreis",DE114,BW,255.177994,-0.930964,6158.588,24134.0,54533.0,38.419998,-0.583964,...,14.963865,18.036458,18.304630,24.950936,24.075167,22.963721,22.949218,20.168262,23.756601,24.578018
08118,2004,"Ludwigsburg, Landkreis",DE115,BW,504.459991,0.341932,15851.325,31422.0,67176.0,47.139999,0.693390,...,15.669149,18.615310,19.154826,24.950936,24.075167,22.963721,22.949218,20.168262,23.756601,24.578018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16075,2012,Saale-Orla-Kreis,DEG0K,TH,84.847000,5.540099,1955.848,23051.0,48013.0,NaN,5.593267,...,NaN,NaN,NaN,23.172179,22.076258,21.326727,21.618260,19.454235,21.870640,22.820639
16076,2012,"Greiz, Landkreis",DEG0L,TH,103.880997,2.040082,1854.354,17851.0,46776.0,NaN,2.091659,...,NaN,NaN,NaN,23.172179,22.076258,21.326727,21.618260,19.454235,21.870640,22.820639
16077,2012,"Altenburger Land, Landkreis",DEG0M,TH,95.289001,-1.616072,1689.681,17732.0,45168.0,NaN,-1.566608,...,NaN,NaN,NaN,23.172179,22.076258,21.326727,21.618260,19.454235,21.870640,22.820639


<a class="anchor" id="8. Difference in differences implementation"></a>

---
# gdp


##  gdp - toas

In [9]:
# use ln_gdp_per_hour_per_employed  or just gdp_per_hour_per_employed in with control did regression??
exog_vars = ['affected_ratio_after',"gva_agr_weight","gva_man_weight","ln_res","ln_fixed_total"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gdp_level,exog,entity_effects=True,time_effects=True)
res_fe_ln_gdp_level = mod.fit(cov_type = 'clustered',cluster_entity=False)




exog_vars = ['affected_ratio_after',"gva_agr_weight","gva_man_weight","ln_fixed_total"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gdp_per_inhabitant,exog,entity_effects=True,time_effects=True)
res_fe_ln_gdp_per_inhabitant = mod.fit(cov_type = 'clustered',cluster_entity=False)




exog_vars = ['affected_ratio_after',"gva_agr_weight","gva_man_weight","ln_res","ln_fixed_total"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gdp_per_employee,exog,entity_effects=True,time_effects=True)
res_fe_ln_gdp_per_employee = mod.fit(cov_type = 'clustered',cluster_entity=False)



exog_vars = ['affected_ratio_after',"gva_agr_weight","gva_man_weight","ln_res","ln_fixed_total"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gdp_per_hour_per_employed,exog,entity_effects=True,time_effects=True)
res_fe_ln_gdp_per_hour_per_employed = mod.fit(cov_type = 'clustered',cluster_entity=False)

# gdp growth
exog_vars = ['affected_ratio_after',"gva_agr_weight","gva_man_weight","ln_res","ln_fixed_total"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gdp,exog,entity_effects=True,time_effects=True)
res_fe_ln_gdp = mod.fit(cov_type = 'clustered',cluster_entity=False)





In [10]:
print(compare({"ln_gdp_level": res_fe_ln_gdp_level, "ln_gdp_per_inhabitant": res_fe_ln_gdp_per_inhabitant, "ln_gdp_per_employee": res_fe_ln_gdp_per_employee}, stars = True, precision = "std_errors"))

                                      Model Comparison                                     
                             ln_gdp_level     ln_gdp_per_inhabitant     ln_gdp_per_employee
-------------------------------------------------------------------------------------------
Dep. Variable                ln_gdp_level     ln_gdp_per_inhabitant     ln_gdp_per_employee
Estimator                        PanelOLS                  PanelOLS                PanelOLS
No. Observations                     2655                      2655                    2655
Cov. Est.                       Clustered                 Clustered               Clustered
R-squared                          0.4574                    0.4563                  0.4469
R-Squared (Within)                 0.2837                    0.2455                  0.4465
R-Squared (Between)                0.5400                   -0.1231                 -0.6432
R-Squared (Overall)                0.5372                   -0.1049             

In [11]:
print(compare({"ln_gdp_per_hour_per_employed": res_fe_ln_gdp_per_hour_per_employed, "ln_gdp": res_fe_ln_gdp,}, stars = True, precision = "std_errors"))

                            Model Comparison                            
                             ln_gdp_per_hour_per_employed         ln_gdp
------------------------------------------------------------------------
Dep. Variable                ln_gdp_per_hour_per_employed         ln_gdp
Estimator                                        PanelOLS       PanelOLS
No. Observations                                     2556           2655
Cov. Est.                                       Clustered      Clustered
R-squared                                          0.4387         0.1309
R-Squared (Within)                                 0.3730         0.1895
R-Squared (Between)                               -1.0497        -108.79
R-Squared (Overall)                               -0.8982        -5.5268
F-statistic                                        353.05         70.726
P-value (F-stat)                                   0.0000         0.0000
======================     ========================

<a class="anchor" id="8. Difference in differences implementation"></a>

---
# gva

##  gva - toas

In [12]:


exog_vars = ['affected_ratio_after',"gva_agr_weight","gva_man_weight","ln_res","ln_fixed_total"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gva_level,exog,entity_effects=True,time_effects=True)
res_fe_ln_gva_level = mod.fit(cov_type = 'clustered',cluster_entity=False)


exog_vars = ['affected_ratio_after',"gva_agr_weight","gva_man_weight","ln_res","ln_fixed_agr"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gva_agr,exog,entity_effects=True,time_effects=True)
res_fe_ln_gva_agr = mod.fit(cov_type = 'clustered',cluster_entity=False)




exog_vars = ['affected_ratio_after',"gva_agr_weight","gva_man_weight","ln_res","ln_fixed_man"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gva_man,exog,entity_effects=True,time_effects=True)
res_fe_ln_gva_man = mod.fit(cov_type = 'clustered',cluster_entity=False)



exog_vars = ['affected_ratio_after',"gva_agr_weight","gva_man_weight","ln_res","ln_fixed_ser"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gva_ser,exog,entity_effects=True,time_effects=True)
res_fe_ln_gva_ser = mod.fit(cov_type = 'clustered',cluster_entity=False)



print(compare({"ln_gva_level": res_fe_ln_gva_level, "ln_gva_agr": res_fe_ln_gva_agr,"ln_gva_man": res_fe_ln_gva_man, "ln_gva_ser": res_fe_ln_gva_ser}, stars = True, precision = "std_errors"))

                                   Model Comparison                                   
                             ln_gva_level     ln_gva_agr     ln_gva_man     ln_gva_ser
--------------------------------------------------------------------------------------
Dep. Variable                ln_gva_level     ln_gva_agr     ln_gva_man     ln_gva_ser
Estimator                        PanelOLS       PanelOLS       PanelOLS       PanelOLS
No. Observations                     2655           2655           2655           2655
Cov. Est.                       Clustered      Clustered      Clustered      Clustered
R-squared                          0.4574         0.2996         0.8813         0.1612
R-Squared (Within)                 0.2994         0.4381         0.6835         0.0456
R-Squared (Between)                0.5400         0.3648         0.5951         0.5735
R-Squared (Overall)                0.5375         0.3664         0.5971         0.5700
F-statistic                        395.72  

In [13]:


exog_vars = ['affected_ratio_after',"emp_agr_weight","emp_man_weight","emp_ser_weight","ln_res","ln_fixed_ser_fin"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gva_ser_fin,exog,entity_effects=True,time_effects=True)
res_fe_ln_gva_ser_fin = mod.fit(cov_type = 'clustered',cluster_entity=False)


exog_vars = ['affected_ratio_after',"emp_agr_weight","emp_man_weight","emp_ser_weight","ln_res","ln_fixed_ser_trade"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gva_ser_trade,exog,entity_effects=True,time_effects=True)
res_fe_ln_gva_ser_trade = mod.fit(cov_type = 'clustered',cluster_entity=False)



exog_vars = ['affected_ratio_after',"emp_agr_weight","emp_man_weight","emp_ser_weight","ln_res","ln_fixed_ser_public"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gva_ser_public,exog,entity_effects=True,time_effects=True)
res_fe_ln_gva_ser_public = mod.fit(cov_type = 'clustered',cluster_entity=False)

print(compare({"ln_gva_ser_fin": res_fe_ln_gva_ser_fin, "ln_gva_ser_trade": res_fe_ln_gva_ser_trade, "ln_gva_ser_public": res_fe_ln_gva_ser_public,}, stars = True, precision = "std_errors"))


                                   Model Comparison                                   
                             ln_gva_ser_fin     ln_gva_ser_trade     ln_gva_ser_public
--------------------------------------------------------------------------------------
Dep. Variable                ln_gva_ser_fin     ln_gva_ser_trade     ln_gva_ser_public
Estimator                          PanelOLS             PanelOLS              PanelOLS
No. Observations                       2673                 2673                  2673
Cov. Est.                         Clustered            Clustered             Clustered
R-squared                            0.0381               0.1936                0.1473
R-Squared (Within)                   0.0627               0.1244               -0.1744
R-Squared (Between)                  0.3957               0.7409                0.6085
R-Squared (Overall)                  0.3922               0.7372                0.6010
F-statistic                          15.573

In [14]:
print(":D")

:D
